# Chiclet Diagram

What was the forecast skill of S2S models? 
Did grid-cell or spatial bias correction schemes improve results?

In [ ]:
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime
import paraguayfloodspy.visualize as viz # custom plotting library

Define parameters

In [ ]:
%matplotlib inline
savefigs = True

Load some plotting parameters from file

In [ ]:
%run ../config/PlotParameters.py
print(extent)
print(cmap)

In [ ]:
area_avg = xr.open_dataset("../data/accessed/AreaAvg.nc")
area_avg['L'] = np.int_(area_avg['L'].astype('timedelta64[D]') / np.timedelta64(1, 'D'))

In [ ]:
df = area_avg.to_dataframe().dropna()
df = df.reset_index()
df['target_date'] = df['S'] + df['L'].apply(np.ceil).apply(lambda x: pd.Timedelta(x, unit='D'))
df = df.loc[np.logical_and(df['target_date'] >= '2015-11-01', df['target_date'] <= '2016-02-29')]
df = df.set_index(['target_date', 'L', 'M'])

In [ ]:
prcp_rpy = pd.read_csv("../data/derived/rainfall_rpy.csv", index_col='time', parse_dates=True)[['raw']]
climatology = np.mean(prcp_rpy).values

In [ ]:
df['tpa'] = df['tp'] - climatology

In [ ]:
forecast = df.to_xarray()['tpa'].mean(dim='M')
ft, fl = np.meshgrid(forecast.target_date, forecast.L)

In [ ]:
fig,axes = plt.subplots(ncols=1, nrows=2, figsize=((15,8)), gridspec_kw = {'height_ratios':[3, 1]})
# Chiclet
ax=axes[0]
C1 = ax.pcolormesh(ft, fl, np.ma.masked_invalid(forecast.transpose().values),
                  vmin=-15, vmax=15, cmap = cmap['rain_a'])
ax.set_xlabel("Target Date")
ax.set_ylabel("Lead Time (Days)")
current_x_lim = ax.get_xlim()
ax.grid(True)
# Observed
ax = axes[1]
ax.plot(prcp_rpy.index.values, prcp_rpy.raw.values)
ax.set_xlim(current_x_lim)
ax.grid(True)
ax.set_ylim(0, 40)
ax.invert_yaxis()
ax.set_ylabel("Rainfall [mm/day]")
# Colorbars
fig.tight_layout()
fig.subplots_adjust(right=0.95)
cax1 = fig.add_axes([0.97, 0.4, 0.01, 0.55])
cbar1 = fig.colorbar(C1, cax = cax1)
cbar1.set_label("Ensemble-Mean Rainfall Forecast [mm/day]")
if savefigs:
    fig.savefig("../figs/Chiclet.pdf", bbox_inches="tight")